# Binance Data

In [4]:
import config, csv
from binance.client import Client

In [7]:
client = Client(config.BINANCE_API_KEY, config.BINANCE_API_SECRET)

In [8]:
dataStart = '1 Jan, 2021'
dataEnd = '2 Jan, 2021'
datafile = 'data/BTCUSDT-20210101-20210102-1m.csv'
csvfile = open(datafile, 'w', newline='')
candlestick_writer = csv.writer(csvfile, delimiter=',')

candlesticks = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1MINUTE, dataStart, dataEnd)

for candlestick in candlesticks:
    candlestick[0] = candlestick[0] / 1000 # divide timestamp to ignore miliseconds
    candlestick_writer.writerow(candlestick)

csvfile.close()

print(datafile, len(candlesticks))

data/BTCUSDT-20210101-20210102-1m.csv 1441


In [11]:
candlesticks[:2]

[[1609459200.0,
  '28923.63000000',
  '28961.66000000',
  '28913.12000000',
  '28961.66000000',
  '27.45703200',
  1609459259999,
  '794382.04398665',
  1292,
  '16.77719500',
  '485390.82682460',
  '0'],
 [1609459260.0,
  '28961.67000000',
  '29017.50000000',
  '28961.01000000',
  '29009.91000000',
  '58.47750100',
  1609459319999,
  '1695802.89696884',
  1651,
  '33.73381800',
  '978176.46820208',
  '0']]

In [9]:
import backtest, csv, os

commission_val = 0.0075 # spot% taker fees binance usdt futures
start_portfolio = 100000.0
portfolio = start_portfolio
stake_val = 1
quantity = 0.10

xchange = 'BTCUSDT'
start = '2021-01-01'
end = '2021-01-02'
strategy = 'RSI'
period = 12
stopLoss = 0
limits = [70,30]
subdir = 'data2'

dataname = 'result/{}-{}-{}-{}.csv'.format(xchange, start, end, '1m')
csvfile = open(dataname, 'w', newline='', encoding='UTF8')
result_writer = csv.writer(csvfile, delimiter=',')
    
for data in os.listdir("./"+subdir):

    plot = True
    if not (start.replace('-','') in data and end.replace('-','') in data): 
        continue
        
    datapath = subdir + '/' + data
    sep = datapath[len(subdir)+1:-4].split(sep='-') # ignore name file 'data/' and '.csv'
    # sep[0] = xchange  sep[1] = start  sep[2] = end  sep[3] = timeframe

    print('\n ------------ ', datapath)
            
               
    end_val, totalwin, totalloss, pnl_net, sqn = backtest.runbacktest(datapath, start, end, period, strategy, \
                                                                      limits[0], limits[1], commission_val, portfolio, stake_val, quantity, stopLoss, plot)

    profit = (pnl_net / portfolio) * 100

    # view the data in the console while processing
    print('%s, %s (Pd %d) (SL %.1f%%) (U%d L%d) Net $%.2f (%.2f%%) WL %d/%d SQN %.2f' % (datapath[5:], strategy, period, stopLoss*100, limits[0], limits[1], end_val - start_portfolio, (end_val - start_portfolio)/start_portfolio*100, totalwin, totalloss, sqn))

    result_writer.writerow([sep[1], strategy, period, stopLoss*100, limits[0], limits[1], "{:.2f}".format(end_val - start_portfolio), totalwin, totalloss, sqn])


csvfile.close()

FileNotFoundError: [Errno 2] No such file or directory: 'result/BTCUSDT-2021-01-01-2021-01-02-1m.csv'

# Best Strategies

In [1]:
import pandas as pd
import os